In [1]:
import torch

In [6]:
data = torch.zeros((16, 6), requires_grad=True)
print(data)

tensor([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]], requires_grad=True)


In [45]:


common_representation = torch.tensor([[1., 0., 0., 0., 0., 0.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 3., 0., 0., 0.],
        [0., 0., 0., 4., 0., 0.],
        [0., 0., 0., 0., 5., 0.],
        [0., 0., 0., 0., 0., 6.],
        [7., 0., 0., 0., 0., 0.],
        [0., 8., 0., 0., 0., 0.],
        [0., 0., 9., 0., 0., 0.],
        [0., 0., 0., 10., 0., 0.],
        [0., 0., 0., 0., 11., 0.],
        [0., 0., 0., 0., 0., 12.],
        [13., 0., 0., 0., 0., 0.],
        [0., 14., 0., 0., 0., 0.],
        [0., 0., 15., 0., 0., 0.],
        [0., 0., 0., 16., 0., 0.]], requires_grad=True)


In [2]:
import time
import torch

# 200个一类
featureSet_size = 200
training_samples_size = 1600
other_featureSet_size = training_samples_size - featureSet_size
common_representation = torch.rand((training_samples_size, 64), device="cuda:1", requires_grad=True)



weight = (featureSet_size+other_featureSet_size)/featureSet_size/other_featureSet_size

In [6]:
start = time.time()

contrastive_loss = torch.tensor(0, dtype=torch.float32)
for i, sample in enumerate(common_representation):
    label_i = int(i / featureSet_size)

    for j, other_sample in enumerate(common_representation):
        if i == j:
            continue
        label_j = int(j / featureSet_size)
        
#         print("the {} and {}".format(i, j))

        # 最小化loss, 异类loss值为正,同类loss值为负
        if label_i == label_j:
            # 对应位置相乘,再求和
#             print("1: {}".format(torch.sum(sample * other_sample)))
#             print("2: {}".format(sample@other_sample))

            contrastive_loss -= torch.sum(sample * other_sample) / featureSet_size
        else:
            contrastive_loss += torch.sum(sample * other_sample) / (training_samples_size-featureSet_size)
#             print("1: {}".format(torch.sum(sample * other_sample)))
#             print("2: {}".format(sample@other_sample))
#     print(contrastive_loss)

contrastive_loss /= common_representation.size()[0]
print(contrastive_loss)
end = time.time()
print(str(end - start))

tensor(0.0795, grad_fn=<DivBackward0>)
447.4881558418274


## 应用元素相乘

In [12]:
start = time.time()

contrastive_loss = torch.tensor(0, dtype=torch.float32)
for i, sample in enumerate(common_representation):
    label_i = int(i / featureSet_size)
    label_list = [i+featureSet_size * label_i for i in range(featureSet_size)]
    all_samples = torch.sum(sample * common_representation) / other_featureSet_size
    class_samples = torch.sum(sample * common_representation[label_list]) *  weight
    self_samples = torch.sum(sample * sample) / featureSet_size
    contrastive_loss += all_samples - class_samples + self_samples   
contrastive_loss /= common_representation.size()[0]
print(contrastive_loss)

end = time.time()
print(str(end - start))

tensor(0.0805, grad_fn=<DivBackward0>)
0.7855644226074219


## 使用@

In [15]:
start = time.time()

contrastive_loss = torch.tensor(0, dtype=torch.float32)
for i, sample in enumerate(common_representation):
    label_i = int(i / featureSet_size)
    label_list = [i+featureSet_size * label_i for i in range(featureSet_size)]
    all_samples = torch.sum(common_representation@sample.t()) / other_featureSet_size
    class_samples = torch.sum(common_representation[label_list]@sample.t()) *  weight
    self_samples = sample@sample.t() / featureSet_size
    contrastive_loss += all_samples - class_samples + self_samples   
contrastive_loss /= common_representation.size()[0]
print(contrastive_loss)

end = time.time()
print(str(end - start))

tensor(0.0805, grad_fn=<DivBackward0>)
0.9034080505371094


## 使用matmul

In [26]:
start = time.time()

contrastive_loss = torch.tensor(0, dtype=torch.float32)
for i, sample in enumerate(common_representation):
    label_i = int(i / featureSet_size)
    label_list = [i+featureSet_size * label_i for i in range(featureSet_size)]
    all_samples = torch.sum(torch.matmul(common_representation, sample.t())) / other_featureSet_size
    
    class_samples = torch.sum(torch.matmul(common_representation[label_list], sample.t())) *  weight
    self_samples = torch.matmul(sample,sample.t()) / featureSet_size
    contrastive_loss += all_samples - class_samples + self_samples   
contrastive_loss /= common_representation.size()[0]
print(contrastive_loss)

end = time.time()
print(str(end - start))

tensor(0.0805, grad_fn=<DivBackward0>)
0.8521888256072998


In [21]:
start = time.time()

contrastive_loss = torch.tensor(0, dtype=torch.float32)
for i, sample in enumerate(common_representation):
    label_i = int(i / featureSet_size)
    
    label_list = [i+featureSet_size * label_i for i in range(featureSet_size)]
    
#     print(sample * common_representation)
    all_samples = torch.sum(sample * common_representation) / other_featureSet_size
#     print((sample * common_representation[label_list]))
    class_samples = torch.sum(sample * common_representation[label_list]) *  weight
#     print(sample * sample)
    self_samples = torch.sum(sample * sample) / featureSet_size
    contrastive_loss += all_samples - class_samples + self_samples   
#     print(contrastive_loss)
contrastive_loss /= common_representation.size()[0]
print(contrastive_loss)

end = time.time()
print(str(end - start))

tensor(0.0805, grad_fn=<DivBackward0>)
0.721799373626709


In [25]:
a = torch.ones(3,4)
b = torch.Tensor([1,2,3,4])
c = a * b
print(c)
torch.sum(c)

tensor([[1., 2., 3., 4.],
        [1., 2., 3., 4.],
        [1., 2., 3., 4.]])


tensor(30.)